# Homework 7 - Risk Premia
#### Group C 26 - Juan Ramirez, Madison Rusch, Tim Taylor, Vidhan Ajmera 

## Question 1 - GMO

1. **GMO’s approach.**

(a) *Why does GMO believe they can more easily predict long-run than short-run asset class performance?*

GMO believes that in the long run, asset classes revert to their fundamental value. In the short term, GMO argues that the market is a 'voting machine' that cares more about the popularity of a certain asset rather than its inherent value.

(b) *What predicting variables does the case mention are used by GMO? Does this fit with the goal of long-run forecasts?*

Return is decomposed into dividend yield, change in P/E ratio, change in profit margin, change in sales. They supplement this data with US macro data on inflation, recessions, GDP, and corporate profits. They look for differences in fundamental value between their view and the market's. This is a very long-term view, because in the short-run returns can occur without reflecting this fundamental value.

(c) *How has this approach led to contrarian positions?*

Sometimes assets can be mispriced for long periods of time, meaning that GMO may sacrifice short-term returns for long-term gains when the market eventually reverts back to pricing based on asset's fundamental value.

(d) *How does this approach raise business risk and managerial career risk?*

Investors are impatient. If they believe that the manager is wrong, or are deeply uncomfortable with short-term losses, then they will withdraw their money and move their business elsewhere. Because it may take years for the market to correct, GMO risks angering investors and losing clients due to low immediate returns.

2. **The market environment.**

(a) *We often estimate the market risk premium by looking at a large sample of historic data. What reasons does the case give to be skeptical that the market risk premium will be as high in the future as it has been over the past 50 years?*

GMO believes that companies will struggle to generate the massive profits of the pre-2009 period, leading to lower valuations. Additionally, in the 2002-2011 period, stocks underperformed short-term bonds, suggesting that companies are struggling to return previous profits.

(b) *In 2007, GMO forecasts real excess equity returns will be negative. What are the biggest drivers of their pessimistic conditional forecast relative to the unconditional forecast. (See Exhibit 9.)*

A falling P/E ratio and lower profit margin. This is likely driven by either lower share prices or earnings stemming from weaker price growth.

(c) *In the 2011 forecast, what components has GMO revised most relative to 2007? Now how does their conditional forecast compare to the unconditional? (See Exhibit 10.)*

P/E ratio growth was said to be zero rather than negative. The 2011 predictions of conditional and unconditional are more similar, but short-run returns are still much lower.

3. **Consider the asset class forecasts in Exhibit 1.**

(a) *Which asset class did GMO estimate to have a negative 10-year return over 2002-2011?*

U.S. Equities

(b) *Which asset classes substantially outperformed GMO’s estimate over that time period?*


Foreign government bonds, emerging market equities, TIPS.

(c) *Which asset classes substantially underperformed GMO’s estimate over that time period?*

US Real Estate Investment Trusts, US Treasury Bills.

4. **Fund Performance.**

(a) *In which asset class was GMWAX most heavily allocated throughout the majority of 1997-2011?*

US Fixed Income.

(b) *Comment on the performance of GMWAX versus its benchmark. (No calculation needed; simply comment on the comparison in the exhibits.)*

GWMAX seemed to earn similar positive returns while staving off a lot of the losses incurred by the benchmark.


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt
import scipy
import statsmodels.api as sm
import scipy.stats as stats
from statsmodels.regression.rolling import RollingOLS
import warnings
warnings.filterwarnings("ignore")

file = "gmo_analysis_data.xlsx"
file.lower()
df = pd.read_excel(file)

signals = pd.read_excel(file, sheet_name='signals',index_col=0)
returns = pd.read_excel(file, sheet_name='returns (total)',index_col=0)
rf = pd.read_excel(file, sheet_name='risk-free rate',index_col=0)

In [2]:
spy = pd.read_excel(file, sheet_name='returns (total)',index_col=0)

## Question 2 - Analysing GMO

1. Calculate the mean, volatility, and Sharpe ratio for GMWAX. Do this for three samples:
- from inception through 2011
- 2012-present
- inception - present

Has the mean, vol, and Sharpe changed much since the case?

In [3]:
returns = returns.subtract(rf['US3M'], axis=0).dropna()

In [4]:
def summStats(data, ann = 12):
    summary_stats = pd.DataFrame(data = None)
    summary_stats['Mean'] = data.mean()*ann
    summary_stats['Volatility'] = data.std(skipna=True)*(ann**(1/2))
    summary_stats['Sharpe'] = summary_stats['Mean']/summary_stats['Volatility']
    return summary_stats

In [5]:
summStats(returns[['GMWAX']][:'2011'])

,Mean,Volatility,Sharpe
GMWAX,0.015827,0.125011,0.126603


In [6]:
summStats(returns[['GMWAX']]['2012':])

,Mean,Volatility,Sharpe
GMWAX,0.036635,0.091996,0.398229


In [7]:
summStats(returns[['GMWAX']])

,Mean,Volatility,Sharpe
GMWAX,0.024497,0.112314,0.218112


2. **GMO believes a risk premium is compensation for a security’s tendency to lose money at “bad times”. For all three samples, analyze extreme scenarios by looking at**
- Min return
- 5th percentile (VaR-5th)
- Maximum drawdown

In [8]:
def maximumDrawdown(returns):
        cum_returns = (1 + returns).cumprod()
        rolling_max = cum_returns.cummax()
        drawdown = (cum_returns - rolling_max) / rolling_max

        max_drawdown = drawdown.min()
        end_date = drawdown.idxmin()
        summary = pd.DataFrame({'Max Drawdown': max_drawdown, 'Bottom': end_date})

        for col in drawdown:
            summary.loc[col,'Peak'] = (rolling_max.loc[:end_date[col],col]).idxmax()
            recovery = (drawdown.loc[end_date[col]:,col])
            try:
                summary.loc[col,'Recover'] = pd.to_datetime(recovery[recovery >= 0].index[0])
            except:
                summary.loc[col,'Recover'] = pd.to_datetime(None)

            summary['Peak'] = pd.to_datetime(summary['Peak'])
            try:
                summary['Duration (to Recover)'] = (summary['Recover'] - summary['Peak'])
            except:
                summary['Duration (to Recover)'] = None

            summary = summary[['Max Drawdown','Peak','Bottom','Recover','Duration (to Recover)']]

        return summary  
    
def riskSummary(returns):
    summary = maximumDrawdown(returns)
    summary['Minimum Return'] = returns.min()
    summary['VaR-5th'] = returns.quantile(.05)
    return summary

In [9]:
riskSummary(returns[['GMWAX']][:'2011'])

,Max Drawdown,Peak,Bottom,Recover,Duration (to Recover),Minimum Return,VaR-5th
GMWAX,-0.472946,1997-09-30,2001-09-30,2011-04-30,4960 days,-0.149179,-0.059806


In [10]:
riskSummary(returns[['GMWAX']]['2012':])

,Max Drawdown,Peak,Bottom,Recover,Duration (to Recover),Minimum Return,VaR-5th
GMWAX,-0.226046,2021-05-31,2022-09-30,NaN,None,-0.11865,-0.039686


In [11]:
riskSummary(returns[['GMWAX']])

,Max Drawdown,Peak,Bottom,Recover,Duration (to Recover),Minimum Return,VaR-5th
GMWAX,-0.472946,1997-09-30,2001-09-30,2011-04-30,4960 days,-0.149179,-0.048293


In [12]:
riskSummary(returns[['SPY']][:'2011'])

,Max Drawdown,Peak,Bottom,Recover,Duration (to Recover),Minimum Return,VaR-5th
SPY,-0.560012,2000-03-31,2009-02-28,NaN,None,-0.16557,-0.080224


In [13]:
riskSummary(returns[['SPY']]['2012':])

,Max Drawdown,Peak,Bottom,Recover,Duration (to Recover),Minimum Return,VaR-5th
SPY,-0.248125,2021-12-31,2022-09-30,NaN,None,-0.124734,-0.068658


In [14]:
riskSummary(returns[['SPY']])

,Max Drawdown,Peak,Bottom,Recover,Duration (to Recover),Minimum Return,VaR-5th
SPY,-0.560012,2000-03-31,2009-02-28,2013-03-31,4748 days,-0.16557,-0.080006


(a) *Does GMWAX have high or low tail-risk as seen by these stats?*

Compared to SPY, GMWAX has lower tail risks overall, with a smaller Max Drawdown, Minimum Return, and VaR.

(b) *Does that vary much across the two subsamples?*

In the 2nd sample, GWMAX's tail risks have improved in VaR terms, but worsened in terms of the Max Drawdown.


3. **For all three samples, regress excess returns of GMWAX on excess returns of SPY.**

(a) Report the estimated alpha, beta, and r-squared.


In [15]:
def regression(data, Y_name, X_name, intercept = True, ann = 12):
    y = data[Y_name]
    if intercept == True:
        x = sm.add_constant(data[X_name])
    else:
        x = data[X_name]
    
    regression = sm.OLS(y, x, missing = 'drop').fit()
    df = regression.params.to_frame('Result')
    df.loc[r'$R^{2}$'] = regression.rsquared
    
    if intercept == True:
        df.loc['const'] *= ann
    return df

In [16]:
regression(returns['2011':], 'GMWAX', 'SPY')

,Result
const,-0.030099
SPY,0.553141
$R^{2}$,0.767202


In [17]:
regression(returns[:'2012'], 'GMWAX', 'SPY')

,Result
const,-0.004424
SPY,0.542852
$R^{2}$,0.515254


In [18]:
regression(returns, 'GMWAX', 'SPY')

,Result
const,-0.016989
SPY,0.545600
$R^{2}$,0.577744


(b) Is GMWAX a low-beta strategy? Has that changed since the case?

Yes, and no it hasn't.

(c) Does GMWAX provide alpha? Has that changed across the subsamples?

No, there seems to be an alpha that is very close to zero.

# Question 3 - Forecast Regressions

1. Consider the lagged regression, where the regressor, (X,) is a period behind the target, ($r^{SPY}$).

In [19]:
signals = signals.shift()
signals['SPY'] = spy['SPY']

In [20]:
signals.columns

Index(['DP', 'EP', 'US10Y', 'SPY'], dtype='object')

In [21]:
# Dividend Price Ratio
div_price = regression(signals, 'SPY', 'DP')
div_price

,Result
const,-0.112859
DP,0.009430
$R^{2}$,0.009355


In [22]:
# Earnings Price Ratio
earnings_price = regression(signals, 'SPY', 'EP')
div_price

,Result
const,-0.112859
DP,0.009430
$R^{2}$,0.009355


In [23]:
# 3 Regressors
div_earnings_10y = regression(signals, 'SPY', ['DP', 'EP', 'US10Y'])
div_earnings_10y

,Result
const,-0.179199
DP,0.007979
EP,0.002651
US10Y,-0.000968
$R^{2}$,0.016333


2. For each of the three regressions, let’s try to utilize the resulting forecast in a trading strategy.

For each strategy estimate:
- mean, volatility, Sharpe
- max-drawdown
- market alpha
- market beta
- market Information ratio

In [24]:
# Div-Price Strategy
wt_div = 100*(div_price.loc['const'][0]/12 + div_price.loc['DP'][0] * signals['DP'])
rt_div = wt_div *  signals['SPY']

# Earnings-Price Strategy
wt_earnings = 100*(earnings_price.loc['const'][0]/12 + earnings_price.loc['EP'][0] * signals['EP'])
rt_earnings = wt_earnings *  signals['SPY']

# 3 Regressor Strategy
wt_3reg = 100*(div_earnings_10y.loc['const'][0]/12 + div_earnings_10y.loc['EP'][0] * signals['EP'] \
               + div_earnings_10y.loc['DP'][0] * signals['DP'] + div_earnings_10y.loc['US10Y'][0] * signals['US10Y'])
rt_3reg = wt_3reg *  signals['SPY']

In [41]:
def summStats_reg(data_Y, data_X, ann = 12):
    stats = pd.DataFrame(data = None, index = ['Summary Stats'])
    stats['Mean'] = data_Y.mean()*ann
    stats['Vol'] = data_Y.std() * (12**0.5)
    stats['Sharpe'] = stats['Mean'] / stats['Vol']
    
    y = data_Y
    x = sm.add_constant(data_X.loc[data_Y.index])
    reg = sm.OLS(y,x,missing = 'drop').fit()
    params = reg.params
    
    cum_ret = (1 + data_Y).cumprod()
    rolling_max = cum_ret.cummax()
    drawdown = (cum_ret - rolling_max) / rolling_max
    stats['Max Drawdown'] = drawdown.min()
    
    stats['alpha'] = params[0] * ann
    stats['beta'] = params[1]
    stats['Info Ratio'] = params[0] * ann / reg.resid.std()*np.sqrt(12)
    return stats

In [42]:
summStats_reg(rt_div, signals['SPY'])

,Mean,Vol,Sharpe,Max Drawdown,alpha,beta,Info Ratio
Summary Stats,0.109454,0.148951,0.734833,-0.653026,0.02067,0.861052,3.310622


In [43]:
summStats_reg(rt_earnings, signals['SPY'])

,Mean,Vol,Sharpe,Max Drawdown,alpha,beta,Info Ratio
Summary Stats,0.1078,0.128587,0.83834,-0.382255,0.032247,0.732726,5.747357


In [44]:
summStats_reg(rt_3reg, signals['SPY'])

,Mean,Vol,Sharpe,Max Drawdown,alpha,beta,Info Ratio
Summary Stats,0.125009,0.145568,0.858768,-0.522125,0.045096,0.775013,6.141798


3. GMO believes a risk premium is compensation for a security’s tendency to lose money at “bad times”. Let’s consider risk characteristics.

(a) For both strategies, the market, and GMO, calculate the monthly VaR for π = .05. Just use the quantile of the historic data for this VaR calculation.


In [49]:
VaR = pd.DataFrame([rt_div.quantile(.05), rt_earnings.quantile(.05), rt_3reg.quantile(.05), 
                    signals['SPY'].quantile(.05), 
                    spy['GMWAX'].dropna().quantile(.05)],
                   index = ['DP Strat','EP Strat','3-factor Strat','SPY','GMO'], 
                   columns = ['5% VaR'])
VaR

,5% VaR
DP Strat,-0.052255
EP Strat,-0.054130
3-factor Strat,-0.064156
SPY,-0.073936
GMO,-0.047306


(b) The GMO case mentions that stocks under-performed short-term bonds from 2000-2011. Does the dynamic portfolio above under-perform the risk-free rate over this time?

In [82]:
summStats(rt_div.to_frame('Div-Price')[['Div-Price']]['2000':'2011'])

,Mean,Volatility,Sharpe
Div-Price,0.039333,0.184209,0.213525


In [83]:
summStats(rt_earnings.to_frame('Earnings-Price')[['Earnings-Price']]['2000':'2011'])

,Mean,Volatility,Sharpe
Earnings-Price,0.037271,0.133899,0.278353


In [84]:
summStats(rt_3reg.to_frame('3 Factor')[['3 Factor']]['2000':'2011'])

,Mean,Volatility,Sharpe
3 Factor,0.060801,0.157393,0.386303


In [85]:
summStats(rf['2000':'2011'])

,Mean,Volatility,Sharpe
US3M,0.023062,0.005785,3.986632


(c) Based on the regression estimates, in how many periods do we estimate a negative risk premium?


In [87]:
r_df = rt_3reg.to_frame('3-factor Strat')
r_df['DP Strat'] = rt_div
r_df['EP Strat'] = rt_earnings
r_df['rf'] = rf['US3M']

df_riskprem = pd.DataFrame(data=None, index=[r'% of periods underperforming $r^{f}$'])
for col in r_df.columns[:3]:
    df_riskprem[col] = len(r_df[r_df[col] < r_df['rf']])/len(r_df) * 100
    
df_riskprem

,3-factor Strat,DP Strat,EP Strat
% of periods underperforming $r^{f}$,36.97479,37.254902,37.254902


(d) Do you believe the dynamic strategy takes on extra risk?

From the results seen so far, it does not seem like the strategies take on extra risk. Overall, we see that their risk, measured by their vol and tail risk stats, are similar to the market's and so not esepcially risky. 

# Question 4 - Out-of-Sample Forecasting

Reconsider the problem above, of estimating (1) for x. The reported $R^{2}$ was the in-sample $R^{2}$ -- it examined how well the forecasts fit in the sample from which the parameters were estimated.

**In particular, focus on the case of using both dividend-price and earnings-price as signals.**

1. Report the out-of-sample $R^{2}$. Did this forecasting strategy produce a positive OOS r-squared?

In [98]:
def OOS_R2(data, Y_name, X_column_names, start):
    y = data[Y_name]
    X = sm.add_constant(data[X_column_names])

    forecast_err, null_err = [], []

    for i,j in enumerate(data.index):
        if i >= start:
            currX = X.iloc[:i]
            currY = y.iloc[:i]
            reg = sm.OLS(currY, currX, missing = 'drop').fit()
            null_forecast = currY.mean()
            reg_predict = reg.predict(X.iloc[[i]])
            actual = y.iloc[[i]]
            forecast_err.append(reg_predict - actual)
            null_err.append(null_forecast - actual)
            
    TS_OOS_error_x = (np.array(forecast_err)**2).sum()
    TS_OOS_error_O = (np.array(null_err)**2).sum()
    return 1 - TS_OOS_error_x/TS_OOS_error_O

In [102]:
div_OOS_R2 = OOS_R2(signals, 'SPY', ['DP'], 60)
print(f'The Dividend-Price OOS R2 is {round(div_OOS_R2*100,4)}%')

The Dividend-Price OOS R2 is -0.2424%


In [103]:
earnings_OOS_R2 = OOS_R2(signals, 'SPY', ['EP'], 60)
print(f'The Earnings-Price OOS R2 is {round(earnings_OOS_R2*100,4)}%')

The Earnings-Price OOS R2 is -0.6966%


In [116]:
three_fac_OOS_R2 = OOS_R2(signals, 'SPY', ['DP', 'EP', 'US10Y'], 60)
print(f'The Earnings-Price OOS R2 is {round(three_fac_OOS_R2*100,4)}%')

The Earnings-Price OOS R2 is -3.2191%


Neither forecasting strategy produced a positive $R^{2}$

2. Re-do problem 3.2 using this OOS forecast. How much better/worse is the OOS strategy compared to the in-sample version of 3.2?

In [106]:
def OOS_strat(data, Y_name, X_column_names, start, weight):
    returns = []
    y = data[Y_name]
    X = sm.add_constant(data[X_column_names])

    for i,j in enumerate(data.index):
        if i >= start:
            currX = X.iloc[:i]
            currY = y.iloc[:i]
            reg = sm.OLS(currY, currX, missing = 'drop').fit()
            pred = reg.predict(X.iloc[[i]])
            w = pred * weight
            returns.append((data.iloc[i][Y_name] * w)[0])

    df_strat = pd.DataFrame(data = returns, index = data.iloc[-(len(returns)):].index, columns = ['Strategy Return'])
    return df_strat

In [110]:
OOS_earnings = OOS_strat(signals, 'SPY', ['EP'], 60, 100)

In [111]:
summStats_reg(OOS_earnings['Strategy Return'], signals['SPY'])

,Mean,Vol,Sharpe,Max Drawdown,alpha,beta,Info Ratio
Summary Stats,0.081898,0.165352,0.495294,-0.583693,0.035321,0.543527,2.98765


In [112]:
OOS_div = OOS_strat(signals, 'SPY', ['DP'], 60, 100)

In [113]:
summStats_reg(OOS_div['Strategy Return'], signals['SPY'])

,Mean,Vol,Sharpe,Max Drawdown,alpha,beta,Info Ratio
Summary Stats,0.079751,0.176263,0.452454,-0.551925,-0.006881,1.010945,-1.056455


In [117]:
OOS_3fac = OOS_strat(signals, 'SPY', ['DP', 'EP', 'US10Y'], 60, 100)

In [118]:
summStats_reg(OOS_3fac['Strategy Return'], signals['SPY'])

,Mean,Vol,Sharpe,Max Drawdown,alpha,beta,Info Ratio
Summary Stats,0.113149,0.252174,0.448693,-0.804959,0.071358,0.487678,3.562222


3. Re-do problem 3.3 using this OOS forecast. Is the point-in-time version of the strategy riskier?

In [121]:
VaR_OOS = pd.DataFrame([OOS_earnings['Strategy Return'].quantile(.05), OOS_div['Strategy Return'].quantile(.05), 
                        OOS_3fac['Strategy Return'].quantile(.05),
                    signals['SPY'].quantile(.05), 
                    returns['GMWAX'].quantile(.05)],
                   index = ['Earnings Strat', 'Dividend Strat','3 Factor Strat','SPY','GMO'], 
                   columns = ['5% VaR'])

VaR_OOS

,5% VaR
Earnings Strat,-0.070971
Dividend Strat,-0.072559
3 Factor Strat,-0.073484
SPY,-0.073936
GMO,-0.048293


In [123]:
summStats(OOS_earnings[['Strategy Return']]['2000':'2011'])

,Mean,Volatility,Sharpe
Strategy Return,0.038768,0.195919,0.197877


In [124]:
summStats(OOS_div[['Strategy Return']]['2000':'2011'])

,Mean,Volatility,Sharpe
Strategy Return,-0.010895,0.163221,-0.066749


In [125]:
summStats(OOS_3fac[['Strategy Return']]['2000':'2011'])

,Mean,Volatility,Sharpe
Strategy Return,0.084077,0.328892,0.255636


In [126]:
summStats(rf.loc['2000':'2011'])

,Mean,Volatility,Sharpe
US3M,0.023062,0.005785,3.986632


Only the earnings and 3 factor strategy outperforms the risk free rate, while the dividend strategy generates a negative return.

In [135]:
r_OOS_df = OOS_earnings.rename(columns={"Strategy Return": "Earnings Strategy"})
r_OOS_df['Dividend Strategy'] = OOS_div[['Strategy Return']]
r_OOS_df['3 factor Strategy'] = OOS_3fac[['Strategy Return']]
r_OOS_df['SPY'] = returns[['SPY']]
r_OOS_df['rf'] = rf['US3M']

In [136]:
df_riskprem_OOS = pd.DataFrame(data=None, index=[r'% of periods underperforming $r^{f}$'])
for col in r_OOS_df.columns[:4]:
    df_riskprem_OOS[col] = len(r_OOS_df[r_OOS_df[col] < r_OOS_df['rf']])/len(r_OOS_df) * 100
    
df_riskprem_OOS

,Earnings Strategy,Dividend Strategy,3 factor Strategy,SPY
% of periods underperforming $r^{f}$,38.383838,39.057239,37.373737,40.06734


Overall, we can see that the strategies underperform the risk free rate at about the same rate as SPY, with the 3-factor strategy performing the best overall in terms underperformance. The VaR of all 3 strategies out of sample are better than SPY, if only marginally.